In [1]:
import os, re
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Bidirectional
from keras.layers import GlobalMaxPool1D, Conv1D, MaxPooling1D, Flatten, AveragePooling1D
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import AUC
from keras.models import load_model

Using TensorFlow backend.


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Open and save data and weights

In [3]:
x_tr_1 = np.genfromtxt('data_preproc/x_tr_1.csv')

In [4]:
x_tr_1.shape

(159571, 200)

In [5]:
x_ts_1 = np.genfromtxt('data_preproc/x_ts_1.csv')

In [6]:
x_ts_1.shape

(153164, 200)

In [7]:
x_tr_2 = np.genfromtxt('data_preproc/x_tr_2.csv')

In [8]:
x_tr_2.shape

(159571, 200)

In [9]:
x_ts_2 = np.genfromtxt('data_preproc/x_ts_2.csv')

In [10]:
x_ts_2.shape

(153164, 200)

In [11]:
x_tr_3 = np.genfromtxt('data_preproc/x_tr_3.csv')

In [12]:
x_tr_3.shape

(159571, 200)

In [13]:
x_ts_3 = np.genfromtxt('data_preproc/x_ts_3.csv')

In [14]:
x_ts_3.shape

(153164, 200)

In [15]:
x_tr_4 = np.genfromtxt('data_preproc/x_tr_4.csv')

In [16]:
x_tr_4.shape

(159571, 200)

In [17]:
x_ts_4 = np.genfromtxt('data_preproc/x_ts_4.csv')

In [18]:
x_ts_4.shape

(153164, 200)

In [19]:
emb_matrix_glove = np.genfromtxt('data_preproc/emb_matr_glove.csv')

In [20]:
emb_matrix_glove.shape

(30000, 200)

In [21]:
emb_matrix_ft = np.genfromtxt('data_preproc/emb_matr_ft.csv')

In [22]:
emb_matrix_ft.shape

(30000, 200)

In [23]:
train = pd.read_csv('data/train.csv/train.csv')
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y_tr = train[col].values

# Convolutional Neural Network

In [24]:
max_len = 200
max_feature = 30000

In [31]:
cp = ModelCheckpoint('model_/best_cnn.h5', monitor='val_acc', verbose=1, save_best_only=True)
es = EarlyStopping(verbose=1)
callbacks = [cp, es]

In [32]:
def build_model(emb_matrix):
    """
    Questa funzione crea la struttura del modello e lo compila
    return: il modello
    
    emb_matrix: matrice che inizializza i pesi dell'Embedding layer
   
    """
    
    inp = Input(shape=(None,))
    emb = Embedding(max_feature, 200, input_length=max_len, weights=[emb_matrix])(inp)
    conv = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(emb)
    glbm = GlobalMaxPool1D()(conv)
    drp1 = Dropout(rate=0.3)(glbm)
    d1 = Dense(units=100, activation='relu')(drp1)
    drp2 = Dropout(rate=0.1)(d1)
    out = Dense(6, activation='sigmoid')(drp2)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='acc')])
    
    return model

In [33]:
# fit with x_tr_1: training data without digits and punctuation

model1 = build_model(emb_matrix_glove)

history1_1 = model1.fit(x_tr_1, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 59s 410us/step - loss: 0.0643 - acc: 0.9645 - val_loss: 0.0489 - val_acc: 0.9819

Epoch 00001: val_acc improved from -inf to 0.98194, saving model to model_/best_cnn.h5
Epoch 2/3
143613/143613 [==============================] - 59s 409us/step - loss: 0.0458 - acc: 0.9840 - val_loss: 0.0465 - val_acc: 0.9820

Epoch 00002: val_acc improved from 0.98194 to 0.98199, saving model to model_/best_cnn.h5
Epoch 3/3
143613/143613 [==============================] - 59s 410us/step - loss: 0.0397 - acc: 0.9886 - val_loss: 0.0477 - val_acc: 0.9767

Epoch 00003: val_acc did not improve from 0.98199
Epoch 00003: early stopping


In [34]:
# fit with x_tr_2: without URLs, stopwords and abbreviations

model2 = build_model(emb_matrix_glove)

history1_2 = model2.fit(x_tr_2, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 59s 412us/step - loss: 0.0628 - acc: 0.9664 - val_loss: 0.0495 - val_acc: 0.9822

Epoch 00001: val_acc improved from 0.98199 to 0.98216, saving model to model_/best_cnn.h5
Epoch 2/3
143613/143613 [==============================] - 59s 410us/step - loss: 0.0458 - acc: 0.9838 - val_loss: 0.0484 - val_acc: 0.9791

Epoch 00002: val_acc did not improve from 0.98216
Epoch 3/3
143613/143613 [==============================] - 59s 411us/step - loss: 0.0392 - acc: 0.9886 - val_loss: 0.0492 - val_acc: 0.9761

Epoch 00003: val_acc did not improve from 0.98216
Epoch 00003: early stopping


In [35]:
# fit with x_tr_3: with lemmatization

model3 = build_model(emb_matrix_glove)

history1_3 = model3.fit(x_tr_3, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 487us/step - loss: 0.0659 - acc: 0.9623 - val_loss: 0.0517 - val_acc: 0.9788

Epoch 00001: val_acc did not improve from 0.98216
Epoch 2/3
143613/143613 [==============================] - 69s 484us/step - loss: 0.0473 - acc: 0.9824 - val_loss: 0.0499 - val_acc: 0.9792

Epoch 00002: val_acc did not improve from 0.98216
Epoch 3/3
143613/143613 [==============================] - 69s 482us/step - loss: 0.0413 - acc: 0.9871 - val_loss: 0.0498 - val_acc: 0.9777

Epoch 00003: val_acc did not improve from 0.98216


In [36]:
# fit with x_tr_4: with stemming

model4 = build_model(emb_matrix_glove)

history1_4 = model4.fit(x_tr_4, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 491us/step - loss: 0.0720 - acc: 0.9515 - val_loss: 0.0586 - val_acc: 0.9689

Epoch 00001: val_acc did not improve from 0.98216
Epoch 2/3
143613/143613 [==============================] - 70s 485us/step - loss: 0.0552 - acc: 0.9734 - val_loss: 0.0563 - val_acc: 0.9724

Epoch 00002: val_acc did not improve from 0.98216
Epoch 3/3
143613/143613 [==============================] - 69s 483us/step - loss: 0.0510 - acc: 0.9781 - val_loss: 0.0559 - val_acc: 0.9723

Epoch 00003: val_acc did not improve from 0.98216


In [37]:
# training with x_tr_1: without digits and punctuation

model1_2 = build_model(emb_matrix_ft)

history2_1 = model1_2.fit(x_tr_1, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 490us/step - loss: 0.0683 - acc: 0.9617 - val_loss: 0.0499 - val_acc: 0.9794

Epoch 00001: val_acc did not improve from 0.98216
Epoch 2/3
143613/143613 [==============================] - 72s 500us/step - loss: 0.0447 - acc: 0.9849 - val_loss: 0.0458 - val_acc: 0.9842

Epoch 00002: val_acc improved from 0.98216 to 0.98421, saving model to model_/best_cnn.h5
Epoch 3/3
143613/143613 [==============================] - 82s 573us/step - loss: 0.0383 - acc: 0.9891 - val_loss: 0.0468 - val_acc: 0.9793

Epoch 00003: val_acc did not improve from 0.98421
Epoch 00003: early stopping


In [38]:
# training with x_tr_2: without URLs, stopswords and abbreviation

model2_2 = build_model(emb_matrix_ft)

history2_2 = model2_2.fit(x_tr_2, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 485us/step - loss: 0.0673 - acc: 0.9631 - val_loss: 0.0503 - val_acc: 0.9835

Epoch 00001: val_acc did not improve from 0.98421
Epoch 2/3
143613/143613 [==============================] - 69s 483us/step - loss: 0.0437 - acc: 0.9856 - val_loss: 0.0475 - val_acc: 0.9794

Epoch 00002: val_acc did not improve from 0.98421
Epoch 3/3
143613/143613 [==============================] - 81s 565us/step - loss: 0.0367 - acc: 0.9903 - val_loss: 0.0501 - val_acc: 0.9745

Epoch 00003: val_acc did not improve from 0.98421
Epoch 00003: early stopping


In [39]:
# training with x_tr_3: with lemamtization

model3_2 = build_model(emb_matrix_ft)

history2_3 = model3_2.fit(x_tr_3, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 487us/step - loss: 0.0652 - acc: 0.9647 - val_loss: 0.0499 - val_acc: 0.9798

Epoch 00001: val_acc did not improve from 0.98421
Epoch 2/3
143613/143613 [==============================] - 70s 485us/step - loss: 0.0456 - acc: 0.9836 - val_loss: 0.0503 - val_acc: 0.9773

Epoch 00002: val_acc did not improve from 0.98421
Epoch 00002: early stopping


In [40]:
# fit with x_tr_4: with stemming

model4_2 = build_model(emb_matrix_ft)

history2_4 = model4_2.fit(x_tr_4, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 487us/step - loss: 0.0731 - acc: 0.9514 - val_loss: 0.0601 - val_acc: 0.9655

Epoch 00001: val_acc did not improve from 0.98421
Epoch 2/3
143613/143613 [==============================] - 69s 484us/step - loss: 0.0548 - acc: 0.9739 - val_loss: 0.0564 - val_acc: 0.9721

Epoch 00002: val_acc did not improve from 0.98421
Epoch 3/3
143613/143613 [==============================] - 69s 484us/step - loss: 0.0502 - acc: 0.9791 - val_loss: 0.0562 - val_acc: 0.9706

Epoch 00003: val_acc did not improve from 0.98421


# Model without embedding weights initialization

In [41]:
def build_model_():
    
    inp = Input(shape=(None,))
    emb = Embedding(max_feature, 200, input_length=max_len)(inp)
    conv = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(emb)
    glbm = GlobalMaxPool1D()(conv)
    drp1 = Dropout(rate=0.3)(glbm)
    d1 = Dense(units=100, activation='relu')(drp1)
    drp2 = Dropout(rate=0.1)(d1)
    out = Dense(6, activation='sigmoid')(drp2)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='acc')])
    
    return model

In [42]:
# training with x_tr_1: without digits and punctuation

model1_3 = build_model_()

history3_1 = model1_3.fit(x_tr_1, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 69s 482us/step - loss: 0.0722 - acc: 0.9581 - val_loss: 0.0524 - val_acc: 0.9751

Epoch 00001: val_acc did not improve from 0.98421
Epoch 2/3
143613/143613 [==============================] - 69s 482us/step - loss: 0.0464 - acc: 0.9835 - val_loss: 0.0495 - val_acc: 0.9796

Epoch 00002: val_acc did not improve from 0.98421
Epoch 3/3
143613/143613 [==============================] - 69s 479us/step - loss: 0.0391 - acc: 0.9892 - val_loss: 0.0501 - val_acc: 0.9770

Epoch 00003: val_acc did not improve from 0.98421
Epoch 00003: early stopping


In [43]:
# training with x_tr_1: without digits and punctuation

model2_3 = build_model_()

history3_2 = model2_3.fit(x_tr_2, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 488us/step - loss: 0.0741 - acc: 0.9565 - val_loss: 0.0533 - val_acc: 0.9773

Epoch 00001: val_acc did not improve from 0.98421
Epoch 2/3
143613/143613 [==============================] - 69s 484us/step - loss: 0.0468 - acc: 0.9832 - val_loss: 0.0523 - val_acc: 0.9758

Epoch 00002: val_acc did not improve from 0.98421
Epoch 3/3
143613/143613 [==============================] - 69s 484us/step - loss: 0.0394 - acc: 0.9892 - val_loss: 0.0536 - val_acc: 0.9716

Epoch 00003: val_acc did not improve from 0.98421
Epoch 00003: early stopping


In [44]:
# training with x_tr_1: without digits and punctuation

model3_3 = build_model_()

history3_3 = model3_3.fit(x_tr_3, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 81s 564us/step - loss: 0.0737 - acc: 0.9566 - val_loss: 0.0563 - val_acc: 0.9717

Epoch 00001: val_acc did not improve from 0.98421
Epoch 2/3
143613/143613 [==============================] - 81s 564us/step - loss: 0.0487 - acc: 0.9814 - val_loss: 0.0543 - val_acc: 0.9747

Epoch 00002: val_acc did not improve from 0.98421
Epoch 3/3
143613/143613 [==============================] - 81s 564us/step - loss: 0.0414 - acc: 0.9875 - val_loss: 0.0574 - val_acc: 0.9628

Epoch 00003: val_acc did not improve from 0.98421
Epoch 00003: early stopping


In [45]:
# training with x_tr_1: without digits and punctuation

model4_3 = build_model_()

history3_4 = model4_3.fit(x_tr_4, y_tr, batch_size=128, epochs=3, validation_split=0.1, callbacks=callbacks)

/home/chiarapower/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 70s 484us/step - loss: 0.0768 - acc: 0.9490 - val_loss: 0.0600 - val_acc: 0.9691

Epoch 00001: val_acc did not improve from 0.98421
Epoch 2/3
143613/143613 [==============================] - 75s 524us/step - loss: 0.0560 - acc: 0.9729 - val_loss: 0.0598 - val_acc: 0.9682

Epoch 00002: val_acc did not improve from 0.98421
Epoch 3/3
143613/143613 [==============================] - 82s 571us/step - loss: 0.0510 - acc: 0.9785 - val_loss: 0.0588 - val_acc: 0.9648

Epoch 00003: val_acc did not improve from 0.98421
